# Glassdoor Scattertext

### This version of the glassdoor scatter text places the pros and cons in one column so that all comments are shown.

In [53]:
import pandas as pd

from datetime import datetime

import spacy

import scattertext as st

import en_core_web_sm

nlp = en_core_web_sm.load()

from string import digits

import re

from IPython.display import HTML
from IPython.display import IFrame

# Load library
from nltk.corpus import stopwords

# You will have to download the set of stop words the first time
import nltk
nltk.download('stopwords')

#from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

import spacy
nlp = spacy.load('en_core_web_sm')

import scattertext as st

import en_core_web_sm
nlp = en_core_web_sm.load()

import bs4


import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import ast

from collections import Counter



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nirun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
#look & feel
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('Datasets\\glassdoor_full_dataset.csv')
df.head()

,date,star,subject,pros,cons,advice,location,clean_member_type,tenure,employee_status,recommend,outlook,ceo,Career Opportunities,Compensation and Benefits,Culture & Values,Senior Management,Work/Life Balance,star_group
0,2019-11-04,5,Big dynamic company with many opportunities,Fast grows up if you are hard worker,unstable rota and early morning shifts,NaN,London,Team Leader,I have been working at Pret A Manger full-time...,Current Employee,NaN,NaN,NaN,Positive,Positive,Positive,Positive,Neutral,Positive
1,2020-01-20,5,Great Vacation Work,Lots of shifts and flexibility of hours,Management are lax on food waste,NaN,Gatwick,Team Member,I worked at Pret A Manger full-time for less t...,Former Employee,NaN,NaN,NaN,Positive,Positive,Positive,Positive,Positive,Positive
2,2020-01-20,4,Great company,"Great training, benefits and salary",Nothing to say so far,NaN,NaN,Team Leader,I have been working at Pret A Manger full-time,Current Employee,NaN,NaN,NaN,Positive,Positive,Positive,Positive,Neutral,Positive
3,2020-01-16,3,Something Helpful,The worst thing is loss of bonus.,New management created silly rules and people ...,Respect your staff more.They are amazing hard ...,London,Production Team Member,I worked at Pret A Manger full-time for more t...,Former Employee,True,positive,positive,Negative,Negative,Negative,Neutral,Negative,Neutral
4,2020-01-15,3,One day probation,"Lot to learn, busy, new people",Very busy,Be more patient with newbies,Palmers Green,Kitchen Staff,I worked at Pret A Manger full-time for less t...,Former Employee,NaN,NaN,NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral


## New df with pro and con comment in one column appended

In [3]:
pros  = df[['date','star','star_group','pros']].rename(columns = {'pros' : 'scatter_comments' })
pros['polarity'] = 'Positive'

cons  = df[['date','star','star_group','cons']].rename(columns = {'cons' : 'scatter_comments' })
cons['polarity'] = 'Negative'


df_2 = pros.append(cons).reset_index()

#drop spare index column
df_2  = df_2.drop(columns = ['index'])


df_2.head()


,date,star,star_group,scatter_comments,polarity
0,2019-11-04,5,Positive,Fast grows up if you are hard worker,Positive
1,2020-01-20,5,Positive,Lots of shifts and flexibility of hours,Positive
2,2020-01-20,4,Positive,"Great training, benefits and salary",Positive
3,2020-01-16,3,Neutral,The worst thing is loss of bonus.,Positive
4,2020-01-15,3,Neutral,"Lot to learn, busy, new people",Positive


#### Restrict table to 2019 only

In [4]:
df_2['date'] = pd.to_datetime(df_2.date)

In [5]:
df_3 = df_2.loc[(df_2.date >= '20190101') & (df_2.date <= '20191231')]
df_3.head()

,date,star,star_group,scatter_comments,polarity
0,2019-11-04,5,Positive,Fast grows up if you are hard worker,Positive
10,2019-12-30,4,Positive,"Staff discount, flexible hours, friendly team,...",Positive
11,2019-12-28,5,Positive,The pay is higher than the minimum wage,Positive
14,2019-12-17,3,Neutral,Money is good for a food chain.,Positive
15,2019-12-14,4,Positive,"Salary and hourly bonuses, free food on a brea...",Positive


In [6]:
df_3['original_comment'] = df_3.scatter_comments

#### Clean

In [7]:
df_3['scatter_comments'] = df_3['scatter_comments'].map(lambda x: x.replace("'", '') )

# No numbers
df_3['scatter_comments'] = df_3['scatter_comments'].map(lambda x: re.sub('[^a-zA-Z]', ' ', x))

# No special characters
df_3['scatter_comments'] =df_3['scatter_comments'].map(lambda x: re.sub(r'\s+', ' ', x))

# No special characters
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("  ", " ")


In [8]:
df_3["scatter_comments"] = df_3["scatter_comments"].str.lower() 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free food and coffee", "freefoodcoffee")
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free food amp coffee", "freefoodcoffee")
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("food and coffee", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free food", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free food,", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free lunch", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("freefood break", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("Free lunch", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free freefood", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("staff food", "freefoodcoffee") 

df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free hot drinks", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("food coffee", "freefoodcoffee") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("free coffee", "freefoodcoffee") 
#df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("coffee", "freefoodcoffee") 

#df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("freefood coffee", "freecoffee") 

df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("bullying", "bully") 

df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("managment", "management") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("managments", "management") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("mystery shopper bonus", "mystery_shopper") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("mystery shopper", "mystery_shopper") 



df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("staff member", "team_member") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("team member", "team_member") 
df_3["scatter_comments"]= df_3["scatter_comments"].str.replace("hard work", "hard_work") 



In [9]:
df_3.loc[df_3.original_comment.str.contains('managment')]

,date,star,star_group,scatter_comments,polarity,original_comment
45,2019-10-20,4,Positive,good benefits good teams good management,Positive,"Good Benefits, good teams, good managment"
48,2019-11-11,1,Negative,bad company with horrible managements,Positive,Bad company with horrible managments
806,2019-08-02,1,Negative,i decided to leave this stress job the money i...,Negative,"I decided to leave this stress job, the money ..."
846,2019-05-21,1,Negative,terrible management,Negative,-terrible managment
859,2019-04-27,1,Negative,bad management constantly understaffed a lot o...,Negative,"Bad managment,constantly understaffed(a lot of..."
866,2019-04-13,1,Negative,poor management harsh work environment very po...,Negative,"poor managment, harsh work environment, very p..."


#### Tokenize

In [10]:
# create a new column that tokenises the comments
df_3['scatter_comment_tokenised'] = df_3.scatter_comments.map(lambda x: nlp(x))
    

#### Lemmatize

In [11]:
# create a function to iterate through the tokenised words and lemmatise them
def lemmatise(sentence):
    lem = []
    for token in sentence:
        lem.append(token.lemma_)
    return lem

In [12]:
#apply function to lemmatise
df_3['scatter_comment_lemm'] = df_3.scatter_comment_tokenised.map(lambda x: lemmatise(x))
df_3.head()

,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm
0,2019-11-04,5,Positive,fast grows up if you are hard_worker,Positive,Fast grows up if you are hard worker,"(fast, grows, up, if, you, are, hard_worker)","[fast, grow, up, if, -PRON-, be, hard_worker]"
10,2019-12-30,4,Positive,staff discount flexible hours friendly team fr...,Positive,"Staff discount, flexible hours, friendly team,...","(staff, discount, flexible, hours, friendly, t...","[staff, discount, flexible, hour, friendly, te..."
11,2019-12-28,5,Positive,the pay is higher than the minimum wage,Positive,The pay is higher than the minimum wage,"(the, pay, is, higher, than, the, minimum, wage)","[the, pay, be, high, than, the, minimum, wage]"
14,2019-12-17,3,Neutral,money is good for a food chain,Positive,Money is good for a food chain.,"(money, is, good, for, a, food, chain)","[money, be, good, for, a, food, chain]"
15,2019-12-14,4,Positive,salary and hourly bonuses freefoodcoffee on a ...,Positive,"Salary and hourly bonuses, free food on a brea...","(salary, and, hourly, bonuses, freefoodcoffee,...","[salary, and, hourly, bonus, freefoodcoffee, o..."


In [13]:
# change list column into a string
df_3['scatter_comment_lemm_str'] = df_3.scatter_comment_lemm.apply(lambda x: ', '.join([str(i) for i in x]))

#replace commas with space bar
df_3['scatter_comment_lemm_str'] = df_3['scatter_comment_lemm_str'].str.replace(',',' ')

df_3.head()

,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm,scatter_comment_lemm_str
0,2019-11-04,5,Positive,fast grows up if you are hard_worker,Positive,Fast grows up if you are hard worker,"(fast, grows, up, if, you, are, hard_worker)","[fast, grow, up, if, -PRON-, be, hard_worker]",fast grow up if -PRON- be hard_worker
10,2019-12-30,4,Positive,staff discount flexible hours friendly team fr...,Positive,"Staff discount, flexible hours, friendly team,...","(staff, discount, flexible, hours, friendly, t...","[staff, discount, flexible, hour, friendly, te...",staff discount flexible hour friendly tea...
11,2019-12-28,5,Positive,the pay is higher than the minimum wage,Positive,The pay is higher than the minimum wage,"(the, pay, is, higher, than, the, minimum, wage)","[the, pay, be, high, than, the, minimum, wage]",the pay be high than the minimum wage
14,2019-12-17,3,Neutral,money is good for a food chain,Positive,Money is good for a food chain.,"(money, is, good, for, a, food, chain)","[money, be, good, for, a, food, chain]",money be good for a food chain
15,2019-12-14,4,Positive,salary and hourly bonuses freefoodcoffee on a ...,Positive,"Salary and hourly bonuses, free food on a brea...","(salary, and, hourly, bonuses, freefoodcoffee,...","[salary, and, hourly, bonus, freefoodcoffee, o...",salary and hourly bonus freefoodcoffee on...


#### Remove Stop Words

In [14]:
# Load stop words
stop = stopwords.words('english')
newStopWords = ['A','also','can', 'Can', 'i', 'dedpends', 'go' ,'too','if','I', 'can', 'it', 'not' , 'nt', 'get', 'you','worked','not','if','lot','per','even','nt','enough','it','always','there','need','lots','part','can','bit','last','things','thing','put','make',"a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]
stop.extend(newStopWords)

df_3['scatter_comments_clean'] = df_3['scatter_comment_lemm_str'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#### Remove Additional words to not parse

In [15]:
do_not_show_list = ['allow','amazing','bad','become','day','depend','due','especially','every','everything','feel','find',
                    'first','friday','full','good','great','happy','high','hot','hour','hourweek','job','know','like','little',
                    'look','many','may','month','move','much','never','nice','often','ok','one','pret','pretmanger','rate',
                    'really','say','see','since','something','sometimes','standard','take','think','want','weeklypay','well',
                    'without','year','-PRON-','stop word','decent','college','working','employee','pret','manger',
                    'pret a manger', 'pret manger', 'Pret manger', 'lovely'

                   ]

In [16]:
def comment_cleaner(sentence):
    
    split_sent = sentence.split()
    
    new_split = []
    
    for word in split_sent:
        
        if word not in do_not_show_list:
            
            new_split.append(word)
            
    new_sentence = ' '.join(new_split)
        
    return new_sentence

In [17]:
df_3['scatter_comments_clean_stop_removed'] = df_3.scatter_comments_clean.map(comment_cleaner)
df_3.head()

,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm,scatter_comment_lemm_str,scatter_comments_clean,scatter_comments_clean_stop_removed
0,2019-11-04,5,Positive,fast grows up if you are hard_worker,Positive,Fast grows up if you are hard worker,"(fast, grows, up, if, you, are, hard_worker)","[fast, grow, up, if, -PRON-, be, hard_worker]",fast grow up if -PRON- be hard_worker,fast grow -PRON- hard_worker,fast grow hard_worker
10,2019-12-30,4,Positive,staff discount flexible hours friendly team fr...,Positive,"Staff discount, flexible hours, friendly team,...","(staff, discount, flexible, hours, friendly, t...","[staff, discount, flexible, hour, friendly, te...",staff discount flexible hour friendly tea...,staff discount flexible hour friendly team fre...,staff discount flexible friendly team fresh or...
11,2019-12-28,5,Positive,the pay is higher than the minimum wage,Positive,The pay is higher than the minimum wage,"(the, pay, is, higher, than, the, minimum, wage)","[the, pay, be, high, than, the, minimum, wage]",the pay be high than the minimum wage,pay high minimum wage,pay minimum wage
14,2019-12-17,3,Neutral,money is good for a food chain,Positive,Money is good for a food chain.,"(money, is, good, for, a, food, chain)","[money, be, good, for, a, food, chain]",money be good for a food chain,money good food chain,money food chain
15,2019-12-14,4,Positive,salary and hourly bonuses freefoodcoffee on a ...,Positive,"Salary and hourly bonuses, free food on a brea...","(salary, and, hourly, bonuses, freefoodcoffee,...","[salary, and, hourly, bonus, freefoodcoffee, o...",salary and hourly bonus freefoodcoffee on...,salary hourly bonus freefoodcoffee break disco...,salary hourly bonus freefoodcoffee break disco...


#### Parse 

In [18]:
# further replacement of words

df_3["scatter_comments_clean_stop_removed"]= df_3["scatter_comments_clean_stop_removed"].str.replace("freefreefood", "freefoodcoffee") 

df_3["scatter_comments_clean_stop_removed"]= df_3["scatter_comments_clean_stop_removed"].str.replace("freefood coffee", "freefoodcoffee") 

df_3["scatter_comments_clean_stop_removed"]= df_3["scatter_comments_clean_stop_removed"].str.replace("freefoodcoffee break", "freefoodcoffee") 

df_3["scatter_comments_clean_stop_removed"]= df_3["scatter_comments_clean_stop_removed"].str.replace("free freefoodcoffee", "freefoodcoffee") 




In [19]:
df_3['new_parsed'] = df_3.scatter_comments_clean_stop_removed.apply(nlp)
df_3.head()

,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm,scatter_comment_lemm_str,scatter_comments_clean,scatter_comments_clean_stop_removed,new_parsed
0,2019-11-04,5,Positive,fast grows up if you are hard_worker,Positive,Fast grows up if you are hard worker,"(fast, grows, up, if, you, are, hard_worker)","[fast, grow, up, if, -PRON-, be, hard_worker]",fast grow up if -PRON- be hard_worker,fast grow -PRON- hard_worker,fast grow hard_worker,"(fast, grow, hard_worker)"
10,2019-12-30,4,Positive,staff discount flexible hours friendly team fr...,Positive,"Staff discount, flexible hours, friendly team,...","(staff, discount, flexible, hours, friendly, t...","[staff, discount, flexible, hour, friendly, te...",staff discount flexible hour friendly tea...,staff discount flexible hour friendly team fre...,staff discount flexible friendly team fresh or...,"(staff, discount, flexible, friendly, team, fr..."
11,2019-12-28,5,Positive,the pay is higher than the minimum wage,Positive,The pay is higher than the minimum wage,"(the, pay, is, higher, than, the, minimum, wage)","[the, pay, be, high, than, the, minimum, wage]",the pay be high than the minimum wage,pay high minimum wage,pay minimum wage,"(pay, minimum, wage)"
14,2019-12-17,3,Neutral,money is good for a food chain,Positive,Money is good for a food chain.,"(money, is, good, for, a, food, chain)","[money, be, good, for, a, food, chain]",money be good for a food chain,money good food chain,money food chain,"(money, food, chain)"
15,2019-12-14,4,Positive,salary and hourly bonuses freefoodcoffee on a ...,Positive,"Salary and hourly bonuses, free food on a brea...","(salary, and, hourly, bonuses, freefoodcoffee,...","[salary, and, hourly, bonus, freefoodcoffee, o...",salary and hourly bonus freefoodcoffee on...,salary hourly bonus freefoodcoffee break disco...,salary hourly bonus freefoodcoffee discount food,"(salary, hourly, bonus, freefoodcoffee, discou..."


#### Scatter Chart

In [20]:
df_3.loc[df_3.original_comment.str.contains('free food!')]

,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm,scatter_comment_lemm_str,scatter_comments_clean,scatter_comments_clean_stop_removed,new_parsed
137,2019-04-16,3,Neutral,nice people and freefoodcoffee,Positive,"Nice people, and free food!","(nice, people, and, freefoodcoffee)","[nice, people, and, freefoodcoffee]",nice people and freefoodcoffee,nice people freefoodcoffee,people freefoodcoffee,"(people, freefoodcoffee)"


In [21]:
df_3.loc[df_3.scatter_comments_clean_stop_removed.str.contains('free freefoodcoffee')]


,date,star,star_group,scatter_comments,polarity,original_comment,scatter_comment_tokenised,scatter_comment_lemm,scatter_comment_lemm_str,scatter_comments_clean,scatter_comments_clean_stop_removed,new_parsed


In [22]:
corpus = st.CorpusFromParsedDocuments(df_3, category_col='polarity', parsed_col='new_parsed').build()

In [203]:
html = st.produce_scattertext_explorer( corpus,
                                        category='Positive',
                                        category_name='Positive Review',
                                        not_category_name='Negative Review',
                                        minimum_term_frequency=5,
                                        filter_unigrams = False,
                                        max_snippets = 400,
                                        #scores = None,
                                        #gray_zero_scores  = True ,
                                        show_top_terms = False,
                                        show_characteristic = False,
                                        #term_significance = None,
                                        width_in_pixels=1000,
                                        #x_coords = None,
                                        #y_coords = None,
                                        #use_full_doc  = True,
                                        #show_axes  = True,
                                        #show_cross_axes = True,
                                        #censor_points   = True,
                                        #topic_model_term_lists  = None,
                                        term_colors = True,
                                        topic_model_preview_size = None,
                                        unified_context   = True, 
                                        #terms_to_include = ['free'] ,
                                        alternative_text_field =  'scatter_comments'
                                        #metadata= df_2['scatter_comments'],
                                        #transform = st.Scalers.log_scale_standardize
                                      )

file_name = 'Pret Glassdoor Sentiment Analysis (2019).html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1400, height=700)

### Remove unwanted elements from html

In [204]:
# load the file
with open("Pret Glassdoor Sentiment Analysis (2019).html") as inf:
    txt = inf.read()
    soup = bs4.BeautifulSoup(txt)
  
 #remove unwanted element from html
soup.find('div', id="d3-div-1-corpus-stats").decompose()

soup.div.append(bs4.BeautifulSoup('<table ><tr><th><img src="https://pret-files.azureedge.net/pretamanger-uk/Doing_The_Right_Thing/Red_heart_retina.jpg" width = "60px"></th><th><h1 style="color:#9f1b32; font-size: 32px;text-shadow: none;letter-spacing: 1px;text-transform: uppercase;display: block;margin-bottom: 23px;padding-top: 25px;">PRET: Glassdoor Sentiment Analysis (2019) </h1></th></tr></table>', 'html.parser'))

#detail
soup.div.append(bs4.BeautifulSoup('''<div><h3 style="color: #404040;text-shadow: none;font-size: 20px;font-weight: normal;line-height: 10px;font-family: "SabonLTStd Roman", "Times New Roman", Times, Serif;letter-spacing: 0.08em;"> This report summarises reviews left by current and former Pret employees the Glassdoor website.</h3></div>''', 'html.parser'))

#final summary
soup.div.append(bs4.BeautifulSoup('''<div><h3 style="color: #404040;text-shadow: none;font-size: 16px;font-weight: normal;line-height: 18px;font-family: "SabonLTStd Roman", "Times New Roman", Times, Serif;letter-spacing: 0.08em;">On Glassdoor employees are asked to leave the pros and cons of working at Pret, we have marked these as positive and negative comments respectively. The graph below summarises these findings by looking at the polarity of the words and how frequently they occur. For example, the word "environment" will be marked neutral as it has the same number of positive and negative comments, whereas the word "stressful" would be completely negative as it only has negative comments. <br><br>The purpose of this graph is to provide a quick summary on the subjects most talked about by our teams. We should use this information to provide an indication of areas we can improve on as an employer in order to create happy teams and consequently happy customers.</h3></div>''', 'html.parser'))

#further detail
soup.div.append(bs4.BeautifulSoup('''<div><h3 style="color: #404040;text-shadow: none;font-size: 16px;font-weight: normal;line-height: 18px;font-family: "SabonLTStd Roman", "Times New Roman", Times, Serif;letter-spacing: 0.08em;">Please note that we have grouped together common terms under one term to make the graph as concise as possible e.g. "free food", "free coffee" and "free food and coffee" have been grouped under the term "freefoodcoffee".</h3></div>''', 'html.parser'))

# legend    
soup.div.append(bs4.BeautifulSoup('''<div><h3 style="text-shadow: none;font-size: 14px;font-weight: normal;line-height: 10px;font-family: "SabonLTStd Roman", "Times New Roman", Times, Serif;letter-spacing: 0.08em;> Legend: <span style="color: #4f82bd"> Positive <span style="color: #000000"> | <span style="color: #f59547"> Neutral <span style="color: #000000"> | <span style="color: #bf504e"> Negative</h3></div>''', 'html.parser'))

#style="color: #404040"
#for div in soup.find_all("div", {'class':'snippet_header'}): 
#    div.decompose()

# save the file again
with open("Pret Glassdoor Sentiment Analysis (2019).html", "w") as outf:
    outf.write(str(soup))